[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pdf-tools/components-code-sample-hub/blob/main/jupyter/pdftools_sdk/pdftools_sdk_pkcs11_sign.ipynb)

In [ ]:
%pip install https://pdftools-public-downloads-production.s3.eu-west-1.amazonaws.com/productkits/PDFSDK/latest/pdftools_sdk-latest.tar.gz
%pip install ipython

# Sign a PDF using a PKCS#11 device
Add a document signature, sometimes called an approval
signature.
This type of signature verifies the integrity of the
signed part of the document and authenticates the
signer's identity.

Validation information is embedded to enable the
long-term validation (LTV) of the signature.

The signing certificate is stored on a cryptographic
device with PKCS#11 middleware (driver).

In [ ]:
import io
from pdftools_sdk.pdf import Document
from pdftools_sdk.sign import Signer
from pdftools_sdk.crypto.providers.pkcs11 import Module, Session

In [ ]:
# Download a file from a given URL and save it to the local system
def prepare_file(url: str, path: str):
    import requests
    response = requests.get(url)
    response.raise_for_status()

    with open(path, 'wb') as f:
        f.write(response.content)

In [ ]:
# Set input arguments
pkcs11_library = 'path/to/pkcs11.dll'  # Placeholder for PKCS#11 library path
password = 'INSERT-PASSWORD'  # Placeholder for PIN
certificate = 'CertificateName'  # Placeholder for certificate name
input_url = 'https://pdftools-public-downloads-production.s3.eu-west-1.amazonaws.com/samples/testfiles/InvoiceNone.pdf'
input_path = 'InvoiceNone.pdf'
prepare_file(input_url, input_path)
output_path = 'SignedPKCS11.pdf'

In [ ]:
def sign(session: Session, certificate: str, input_path: str, output_path: str):
    # Create the signature configuration for the certificate
    signature = session.create_signature_from_name(certificate)

    # Open input document
    with io.FileIO(input_path, 'rb') as in_stream:
        with Document.open(in_stream) as input_document:

            # Create stream for output file
            with io.FileIO(output_path, 'wb+') as output_stream:

                # Sign the input document
                signer = Signer()
                signer.sign(input_document, signature, output_stream)

In [ ]:
try:
    # By default, a test license key is active. In this case, a watermark is added to the output. 
    # If you have a license key, please uncomment the following call and set the license key.
    # from pdftools_sdk.sdk import Sdk
    # Sdk.initialize("<PDFSDK,V1,MGAASQD6L2JMQHL54PK08RQX8GG4SS0M8DAHVPH0VMP3NB8R9DUK>")

    # Optional: Set your proxy configuration
    # Sdk.set_proxy("http://myproxy:8080")
    
    # Load the PKCS#11 driver module (middleware)
    # The module can only be loaded once in the application.
    with Module.load(pkcs11_library) as module:
    
        # Create a session to the cryptographic device and log in with the password (pin)
        # Use devices[i] if you have more than one device installed instead of devices.get_single()
        with module.devices.get_single().create_session(password) as session:
            # Sign a PDF document
            sign(session, certificate, input_path, output_path)

    print(f"Successfully created file {output_path}")
except Exception as e:
    print(f"An error occurred: {e}")